In [1]:
from transformers import VideoMAEForVideoClassification
from transformers import AutoImageProcessor, VideoMAEForPreTraining, VideoMAEConfig
import torch.nn as nn
import torch
import numpy as np


/home/jth264/.conda/envs/benchmark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
model_ckpt = "MCG-NJU/videomae-base" # pre-trained model from which to fine-tune
model = VideoMAEForVideoClassification.from_pretrained(model_ckpt, attn_implementation="sdpa", torch_dtype=torch.float16)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
print(model.classifier.in_features)
print(model.classifier.out_features)

768
2


In [7]:
config = VideoMAEConfig()
custom_classes_classifier = VideoMAEForVideoClassification(config)
custom_classes_classifier.config

VideoMAEConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 4,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 16,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.50.0",
  "tubelet_size": 2,
  "use_mean_pooling": true
}

In [9]:
num_frames = 16
video = list(np.random.randint(0, 256, (num_frames, 3, 224, 224)))

In [10]:
for name, param in model.named_parameters():
    print(name, param.shape)

NameError: name 'model' is not defined

In [11]:
model.classifier = nn.Linear(model.classifier.in_features, 10)
#random initialize weights
nn.init.xavier_uniform_(model.classifier.weight)

NameError: name 'model' is not defined

In [12]:
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
pixel_values = image_processor(video, return_tensors="pt", use_fast = True).pixel_values


In [13]:
type(pixel_values)

torch.Tensor

In [14]:
pixel_values.shape

torch.Size([1, 16, 3, 224, 224])

In [15]:
outputs = custom_classes_classifier(pixel_values)

In [16]:
type(outputs)

transformers.modeling_outputs.ImageClassifierOutput

In [18]:
outputs.logits

tensor([[ 0.6306, -0.0873]], grad_fn=<AddmmBackward0>)

In [2]:
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, VideoMAEForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

/home/jth264/.conda/envs/benchmark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


eating spaghetti


In [39]:
from huggingface_hub import hf_hub_download

hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
destination_path = "/share/j_sun/jth264"  # Change this to your desired directory

file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, 
    filename=filename, 
    repo_type="dataset",
    local_dir=destination_path  # Specify the download directory
)


In [4]:

video_path = '/share/j_sun/jth264/UCF101_subset/test/Basketball/v_Basketball_g02_c04.avi' 
container = av.open(video_path)

In [ ]:
container.streams.video[0].frames

80

In [9]:
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

In [11]:
video.shape

(16, 240, 320, 3)

In [12]:
torch.tensor(video).shape

torch.Size([16, 240, 320, 3])

In [13]:
#rearrage the dimensions
video = torch.tensor(video).permute(0, 3, 1, 2)

In [14]:
video.shape


torch.Size([16, 3, 240, 320])

In [ ]:
data_path = '/share/j_sun/jth264/UCF101_subset'
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
video_transform = lambda video: image_processor(list(video), return_tensors="pt").pixel_values
dataset = UCF101(data_path, transform=video_transform)

In [ ]:
container = av.open(video_path)
clip_length = 16
stride = 1
def sliding_window_sampler(clip_length, stride, container):
    len = container.streams.video[0].frames
    for start in range(0, len - clip_length + 1, stride):
        yield container[start : start + clip_length]

for indices in sliding_window_sampler(clip_length, stride, container):
    print(indices.shape)  # Should print (16, 3, 224, 224)
    input_tensor = read_video_pyav(container, indices)
    pixel_values = image_processor(list(input_tensor), return_tensors="pt").pixel_values
    

In [ ]:
import av
from utils import read_video_pyav
clip_length = 16
video_path = '/share/j_sun/jth264/UCF101_subset/test/Basketball/v_Basketball_g02_c04.avi'
container = av.open(video_path)
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base", use_fast = True)
video_transform = lambda video: image_processor(list(video), return_tensors="pt").pixel_values.squeeze(0)
total_frames = container.streams.video[0].frames
print(f"Total frames in the video: {total_frames}")
video = read_video_pyav(container, list(range(0,container.streams.video[0].frames)))
type(video)
print(video.shape)
inputs = video_transform(video)
print(inputs.shape)

Total frames in the video: 80
(80, 240, 320, 3)
torch.Size([1, 80, 3, 224, 224])
